In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/question-pairs-dataset/questions.csv


In [2]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer


2025-08-24 19:53:18.518532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756065198.873165      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756065198.973556      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv('/kaggle/input/question-pairs-dataset/questions.csv')

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404351 entries, 0 to 404350
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404351 non-null  int64 
 1   qid1          404351 non-null  int64 
 2   qid2          404351 non-null  int64 
 3   question1     404350 non-null  object
 4   question2     404349 non-null  object
 5   is_duplicate  404351 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [6]:
df = df.dropna(subset=['question1','question2','is_duplicate'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 404348 entries, 0 to 404350
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404348 non-null  int64 
 1   qid1          404348 non-null  int64 
 2   qid2          404348 non-null  int64 
 3   question1     404348 non-null  object
 4   question2     404348 non-null  object
 5   is_duplicate  404348 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 21.6+ MB


In [8]:
df['is_duplicate'].value_counts()

is_duplicate
0    255042
1    149306
Name: count, dtype: int64

In [16]:
df_small = df.iloc[0:10000]

In [17]:
q1_list = list[df['question1']]
q2_list = list[df['question2']]

In [18]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [19]:
q1_list

list[0         What is the step by step guide to invest in sh...
1         What is the story of Kohinoor (Koh-i-Noor) Dia...
2         How can I increase the speed of my internet co...
3         Why am I mentally very lonely? How can I solve...
4         Which one dissolve in water quikly sugar, salt...
                                ...                        
404346    How many keywords are there in the Racket prog...
404347            Do you believe there is life after death?
404348                                    What is one coin?
404349    What is the approx annual cost of living while...
404350                What is like to have sex with cousin?
Name: question1, Length: 404348, dtype: object]

In [20]:
q1_embeddings = model.encode(df_small['question1'].to_list(),convert_to_numpy=True)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [38]:
np.concatenate([np.abs(q1_embeddings-q2_embeddings),q1_embeddings*q1_embeddings],axis=1)[1:2].shape

(1, 768)

In [21]:
q2_embeddings = model.encode(df_small['question2'].to_list(),convert_to_numpy=True)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [22]:
from sentence_transformers import util
similarity = util.cos_sim(q1_embeddings, q2_embeddings)  # Returns a tensor


In [24]:
np.concatenate(
        [np.abs(q1_embeddings - q2_embeddings), q1_embeddings * q2_embeddings], axis=1
    ).shape

(10000, 768)

In [22]:
similarity.shape

torch.Size([10000, 10000])



Model

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [11]:
X = df[['question1','question2']]
y = df['is_duplicate']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [14]:
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [59]:
def get_embeddings(df_subset):
    q1_embeds = model.encode(df_subset['question1'].to_list(),convert_to_numpy=True)
    q2_embeds = model.encode(df_subset['question2'].to_list(),convert_to_numpy=True)
    features = np.concatenate(
        [np.abs(q1_embeds - q2_embeds), q1_embeds * q2_embeds], axis=1
    )
    return features

In [60]:
X_train_emb = get_embeddings(X_train)
X_test_emb = get_embeddings(X_test)

Batches:   0%|          | 0/10109 [00:00<?, ?it/s]

Batches:   0%|          | 0/10109 [00:00<?, ?it/s]

Batches:   0%|          | 0/2528 [00:00<?, ?it/s]

Batches:   0%|          | 0/2528 [00:00<?, ?it/s]

In [61]:
X_test_emb.shape

(80870, 768)

In [62]:
y_test.shape

(80870,)

In [63]:
clf = LogisticRegression(max_iter=200)
clf.fit(X_train_emb,y_train)

LogisticRegression(max_iter=200)

In [64]:
pred = clf.predict(X_test_emb)

In [65]:
from sklearn.metrics import accuracy_score, classification_report

In [67]:
acc = accuracy_score(y_test, pred)
print(f"Accuracy: {acc:.4f}")
print("Classification Report:\n", classification_report(y_test, pred))

Accuracy: 0.8084
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.85      0.85     51009
           1       0.74      0.73      0.74     29861

    accuracy                           0.81     80870
   macro avg       0.79      0.79      0.79     80870
weighted avg       0.81      0.81      0.81     80870

